## frathom
## Homework 4 Part1

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as tf
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import helper
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset
import glob
import librosa
import math
import os
import random
import pickle
from itertools import combinations

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
folder_path='/content/gdrive/My Drive/HW4/'

In [4]:
!ls '/content/gdrive/My Drive/HW4/'

hw4_te7.pkl  hw4_tes.pkl  hw4_tr7.pkl  hw4_trs.pkl


In [5]:
folder_list=os.listdir(folder_path)

In [6]:
folder_list

['hw4_trs.pkl', 'hw4_tr7.pkl', 'hw4_te7.pkl', 'hw4_tes.pkl']

In [7]:
for i in folder_list:
  if(i=='hw4_trs.pkl'):
    trs_file= open(folder_path+i,'rb')
    train=pickle.load(trs_file)
    trs_file.close()
      
  if(i=='hw4_tes.pkl'):
    tes_file= open(folder_path+i,'rb')
    test=pickle.load(tes_file)
    tes_file.close()
  

In [8]:
train_s=[]
for i in train:
  sig_train=librosa.stft(i,n_fft=1024,hop_length=512)
  train_s.append(np.abs(sig_train.T))
train_s=np.array(train_s)

In [9]:
test_s=[]
for i in test:
  sig_test=librosa.stft(i,n_fft=1024,hop_length=512)
  test_s.append(np.abs(sig_test.T))
test_s=np.array(test_s)

In [10]:
train_s.shape

(500, 32, 513)

In [11]:
test_s.shape

(200, 45, 513)

In [12]:
def comb(s,e):
  return list(combinations(range(s,e),2))

In [13]:
def repeat(to_pa,data):
  #total_pairs=np.array(to_pa)
  if(data=='train'):
    s1=train_s[total_pairs[:,0]]
    s2=train_s[total_pairs[:,1]]
    labels=np.tile(np.concatenate((np.ones((20,1)), np.zeros((20,1))), axis = 0),(50,1))

  if(data=='test'):
    s1=test_s[total_pairs[:,0]]
    s2=test_s[total_pairs[:,1]]
    labels=np.tile(np.concatenate((np.ones((20,1)), np.zeros((20,1))), axis = 0),(20,1))

  return s1,s2,labels


In [14]:
total=train_s.shape[0]
to_pa=[]
en=0
for i in range(50):
  st=en
  en=(i+1)*10
  co=comb(st,en)
  positive=random.sample(co,20)
  current=list(np.random.choice(range(st,en),20))
  others=random.sample(list(set(range(total))-set(range(st,en))),20)
  negative=list(zip(current,others))
  #to_pa=to_pa.extend(positive)
  #to_pa=to_pa.extend(negative)
  to_pa=to_pa+positive+negative

total_pairs=np.array(to_pa)
s1_train,s2_train,train_labels=repeat(total_pairs,'train')





In [15]:
train_labels=np.reshape(train_labels,(train_labels.shape[0],))
train_labels.shape

(2000,)

In [16]:
total=test_s.shape[0]
to_pa=[]
en=0
for i in range(20):
  st=en
  en=(i+1)*10
  co=comb(st,en)
  positive=random.sample(co,20)
  current=list(np.random.choice(range(st,en),20))
  others=random.sample(list(set(range(total))-set(range(st,en))),20)
  negative=list(zip(current,others))
  #to_pa=to_pa.extend(positive)
  #to_pa=to_pa.extend(negative)
  to_pa=to_pa+positive+negative

total_pairs=np.array(to_pa)
s1_test,s2_test,test_labels=repeat(total_pairs,'test')

In [17]:
test_labels=np.reshape(test_labels,(test_labels.shape[0],))
test_labels.shape

(800,)

In [424]:
class SN(nn.Module):
  def __init__(self,inp_size,hid_size,num_lay,out_size):
    super(SN,self).__init__()
    self.inp_size=inp_size
    self.hid_size=hid_size
    self.num_lay=num_lay
    self.gru=nn.GRU(inp_size,hid_size,num_lay,batch_first=True,dropout=0.4)
    self.lay1=nn.Linear(hid_size,hid_size)
    self.lay2=nn.Linear(hid_size,out_size)
    self.sig=nn.Sigmoid()
    self.tanh=nn.Tanh()
  
  def common_forward(self,inp):
    out,hid=self.gru(inp)
    o=out[:,-1,:].squeeze()
    tanh_out=self.tanh(self.lay2(self.lay1(o)))
    return tanh_out

  def forward(self,inp1,inp2):
    o1=self.common_forward(inp1).unsqueeze(1)
    o2=self.common_forward(inp2).unsqueeze(2)
    out=torch.bmm(o1,o2).squeeze()
    return self.sig(out)

  


In [425]:
model = SN( 513,16,2,150).cuda()
loss = nn.BCELoss()
optimize = torch.optim.Adam(model.parameters(),lr =0.0002)
batch_num=150

In [426]:
for i in range(200):
  for j in range(0,len(s1_train),batch_num):
    s1_tr=torch.FloatTensor(s1_train[j:j+batch_num]).cuda()
    s2_tr=torch.FloatTensor(s2_train[j:j+batch_num]).cuda()
    ytrain=torch.FloatTensor(train_labels[j:j+batch_num]).cuda()
    ff_out=model(s1_tr,s2_tr)
    #print(ff_out.shape)
    losses=loss(ff_out,ytrain)
    optimize.zero_grad()
    losses.backward()
    optimize.step()
  print('epoch {} is done and the loss is {}'.format(i, losses.data))

epoch 0 is done and the loss is 2.1117162704467773
epoch 1 is done and the loss is 1.8169549703598022
epoch 2 is done and the loss is 1.6007604598999023
epoch 3 is done and the loss is 1.5060206651687622
epoch 4 is done and the loss is 1.4025521278381348
epoch 5 is done and the loss is 1.2889007329940796
epoch 6 is done and the loss is 1.1454260349273682
epoch 7 is done and the loss is 1.1219291687011719
epoch 8 is done and the loss is 1.1137303113937378
epoch 9 is done and the loss is 1.0254050493240356
epoch 10 is done and the loss is 0.995309591293335
epoch 11 is done and the loss is 0.9139983654022217
epoch 12 is done and the loss is 0.8536626100540161
epoch 13 is done and the loss is 0.8031637668609619
epoch 14 is done and the loss is 0.8143818378448486
epoch 15 is done and the loss is 0.7893164753913879
epoch 16 is done and the loss is 0.7568760514259338
epoch 17 is done and the loss is 0.756738007068634
epoch 18 is done and the loss is 0.7158694863319397
epoch 19 is done and the

In [463]:
with torch.no_grad():
  s1_te=torch.FloatTensor(s1_test).cuda()
  s2_te=torch.FloatTensor(s2_test).cuda()
  out = model(s1_te, s2_te) 
thres = 0.5
predict = np.zeros((s1_test.shape[0]))
predict[out.detach().cpu().numpy() > thres] = 1
print('Test Accuracy is:{} %'.format((sum(predict == test_labels)/s1_test.shape[0])*100))

Test Accuracy is:66.5 %
